# ID2214/FID3214 Assignment 3 Group no. [2]
### Project members: 
[Farrokh Bolandi, bolandi@kth.se]
[Ezio Cristofoli, ezioc@kth.se]
[Abyel Tesfay, Abyel@kth.se]

### Declaration
By submitting this solution, it is hereby declared that all individuals listed above have contributed to the solution, either with code that appear in the final solution below, or with code that has been evaluated and compared to the final solution, but for some reason has been excluded. It is also declared that all project members fully understand all parts of the final solution and can explain it upon request.

It is furthermore declared that the code below is a contribution by the project members only, and specifically that no part of the solution has been copied from any other source (except for lecture slides at the course ID2214/FID3214) and no part of the solution has been provided by someone not listed as project member above.

It is furthermore declared that it has been understood that no other library/package than the Python 3 standard library, NumPy, pandas, time and sklearn.tree, may be used in the solution for this assignment.

### Instructions
All parts of the assignment starting with number 1 below are mandatory. Satisfactory solutions
will give 1 point (in total). If they in addition are good (all parts work more or less 
as they should), completed on time (submitted before the deadline in Canvas) and according
to the instructions, together with satisfactory solutions of all parts of the assignment starting 
with number 2 below, then the assignment will receive 2 points (in total).

Note that you do not have to develop the code directly within the notebook
but may instead copy the comments and test cases to a more convenient development environment
and when everything works as expected, you may paste your functions into this
notebook, do a final testing (all cells should succeed) and submit the whole notebook 
(a single file) in Canvas (do not forget to fill in your group number and names above).

## Load NumPy, pandas, time and DecisionTreeClassifier from sklearn.tree

In [1]:
import numpy as np
import pandas as pd
import time
from sklearn.tree import DecisionTreeClassifier

## Reused functions from Assignment 1

In [7]:
# Copy and paste functions from Assignment 1 here that you need for this assignment

# Copy and paste functions from Assignment 1 here that you need for this assignment
def create_column_filter(df):
    df2 = df.copy()
    column_filter = list(df2.columns)
    columns = [col for col in df2.columns if col != 'CLASS' and col != 'ID']
    for col in columns:
        if df2[col].isnull().all():
            df2.drop(columns=col, inplace=True)
            column_filter.remove(col)
            continue

        if len(df2[col].dropna().unique()) <= 1:
            df2.drop(columns=col, inplace=True)
            column_filter.remove(col)

    return df2, column_filter


def apply_column_filter(df, column_filter):
    df2 = df.copy()
    [df2.drop(columns=col, inplace=True) for col in df2.columns if col not in column_filter]
    return df2

def create_normalization(df, normalizationtype='minmax'):
    df2 = df.copy()
    include_types = np.int32, np.int64, np.float32, np.float64
    columns = [col for col in df2.columns if (col != 'CLASS' and col != 'ID') 
               and df2[col].dtype in include_types]
    normalization = {}
    for col in columns:
        if normalizationtype=='minmax':
            min = df2[col].min()
            max = df2[col].max()
            normalization[col] = normalizationtype, min, max
        elif normalization=='zscore':
            mean = df2[col].mean()
            std = df[col].std()
            normalization = normalizationtype, mean, std
            
    for col in columns:
        values = list(normalization[col])
        if values[0] == 'minmax':
            df2[col] = [(x-values[1])/(values[2]-values[1]) for x in df[col]]

    return df2, normalization

def apply_normalization(df, normalization):
    df2 = df.copy()
    include_types = np.int32, np.int64, np.float32, np.float64
    columns = [col for col in df2.columns if (col != 'CLASS' and col != 'ID') 
               and df2[col].dtype in include_types]
    for col in columns:
        values = list(normalization[col])
        if values[0] == 'minmax':
            df2[col] = [(x-values[1])/(values[2]-values[1]) for x in df[col]]

    
    return df2

def create_imputation(df):
    df2 = df.copy()
    numeric_types = np.int32, np.int64, np.float32, np.float64
    columns = [col for col in df2.columns if (col != 'CLASS' and col != 'ID')]
    imputation = {}
    for col in columns:
        if df2[col].dtype in numeric_types:
            if df2[col].isnull().all():
                df2[col].fillna(0, inplace=True)
            imputation[col] = df2[col].mean()
            df2[col].fillna(df2[col].mean(), inplace=True)
        else:
            if df2[col].isnull().all():
                df2[col].fillna('', inplace=True) if df2[col].dtype == 'object' else \
                    df2[col].astype('category') and df2[col].fillna(df2[col].cat.categories[0], inplace=True)
            
            imputation[col] = df2[col].mode()[0]
            df2[col].fillna(imputation[col], inplace=True)
        
    return df2, imputation

def apply_imputation(df, imputation):
    df2 = df.copy()
    return df2.fillna(value=imputation)

def create_one_hot(df):
    df2=df.copy()
    columns = [col for col in df2.columns if (col != 'CLASS' and col != 'ID')]
    one_hot={}
    for col in columns:
        if df2[col].dtype.name != 'category' and df2[col].dtype.name != 'object':
            continue
        one_hot[col]=df2[col].unique()
        tmp = pd.get_dummies(df2[col], prefix=col, prefix_sep='-', dtype=np.float64)
        df2.drop(columns=col, inplace=True)
        df2 = pd.concat([df2, tmp], axis=1)

    return df2, one_hot

def apply_one_hot(df, one_hot):
    df2=df.copy()
    columns = [col for col in df2.columns if (col != 'CLASS' and col != 'ID')]
    for col in columns:
        if df2[col].dtype.name != 'category' and df2[col].dtype.name != 'object':
            continue
        tmp = pd.get_dummies(df2[col], prefix=col, prefix_sep='-', dtype=np.float64)
        df2.drop(columns=col, inplace=True)
        df2 = pd.concat([df2, tmp], axis=1)
    return df2    

def accuracy(df, correctlabels):
    highest_probability = df.idxmax(axis=1)
    correct_occurances = 0
    for correct_label, predicted_label in zip(correctlabels, highest_probability):
        if correct_label==predicted_label:
            correct_occurances+=1 
    
    return correct_occurances/df.index.size

def brier_score(df, correctlabels):
    squared_sum = 0
    row = 0
    for label in correctlabels:
        i = np.where(df.columns==label)[0]
        for col in df.columns:
            squared_sum += (1 - df.loc[row, label])**2 if label==col else df.loc[row, col]**2
        row+=1
    
    return squared_sum/df.index.size

def auc(df, correctlabels):
    auc=0
    for col in df.columns:
        df2 = pd.concat([df[col], pd.Series(correctlabels.astype('category'), name='correct')], axis=1)
        # get dummies for correct labels and sort descending
        df2 = pd.get_dummies(df2.sort_values(col, ascending=False))
        
        # move col to first for easier total tp and fp calculation
        tmp=df2.pop('correct_'+str(col))
        # get the col frequency for calculating weighted AUCs
        col_frequency=tmp.sum()/tmp.index.size
        df2.insert(1, tmp.name, tmp)
        scores={}
        # populate the scores dictionary for column i.e. key=score, value=[tp_sum, fp_sum]
        for row in df.index:
            key=df2.iloc[row, 0]
            current=np.zeros(2, dtype=np.uint) if scores.get(key) is None else scores[key]
            to_add=np.array([1,0]) if df2.iloc[row, 1]==1 else np.array([0,1])
            scores[key]=current+to_add

        # calculate auc based on scores
        cov_tp=0
        column_auc=0
        tot_tp=0
        tot_fp=0
        # calculate total tp and fp 
        for value in scores.values():
            tot_tp+=int(value[0])
            tot_fp+=int(value[1])
            
        # same algorithm as in the lecture (bad naming though)
        for i in scores.values():
            if i[1] == 0:
                cov_tp+=i[0]
            elif i[0] == 0:
                column_auc += (cov_tp/tot_tp)*(i[1]/tot_fp)
            else:
                column_auc += (cov_tp/tot_tp)*(i[1]/tot_fp)+(i[0]/tot_tp)*(i[1]/tot_fp)/2
                cov_tp += i[0]

        auc+=col_frequency*column_auc
    
    return auc

def create_bins(df, nobins=10, bintype='equal-width'):
    df2=df.copy()
    include_types = np.int32, np.int64, np.float32, np.float64
    columns = [col for col in df2.columns if (col != 'CLASS' and col != 'ID') 
               and df2[col].dtype in include_types]
    binning ={}
    for col in columns:
        df2[col].astype('category')
        if (bintype == 'equal-width'):
            res, bins = pd.cut(df2[col], nobins, retbins=True, labels=False)
            bins[0] = -np.inf
            bins[len(bins)-1] = np.inf
            binning[col] = bins
        elif (bintype == 'equal-size'):
            # We drop duplicates which results in fewer bins
            res, bins = pd.qcut(df2[col], nobins, retbins=True, labels=False, duplicates='drop')
            bins[0] = -np.inf
            bins[len(bins)-1] = np.inf
            binning[col] = bins
    return df2, binning

def apply_bins(df, binning):
    df2 = df.copy()
    columns = [col for col in df2.columns if (col != 'CLASS' and col != 'ID')] 
    for col in columns:
        labels=list(range(len(binning[col])-1)) 
        df2[col] = pd.cut(df2[col], binning[col], labels=labels)
    return df2

def create_bins(df, nobins=10, bintype='equal-width'):
    df2 = df.copy()
    include_types = np.int32, np.int64, np.float32, np.float64
    columns = [col for col in df2.columns if (col != 'CLASS' and col != 'ID')
               and df2[col].dtype in include_types]
    binning = {}
    for col in columns:
        df2[col].astype('category')
        if (bintype == 'equal-width'):
            df2[col], bins = pd.cut(df2[col], nobins, retbins=True, duplicates='drop', labels=False)
            bins[0] = -np.inf
            bins[len(bins) - 1] = np.inf
            binning[col] = bins
        elif (bintype == 'equal-size'):
            # We drop duplicates which results in fewer bins
            df2[col], bins = pd.qcut(df2[col], nobins, retbins=True, labels=False, duplicates='drop')
            bins[0] = -np.inf
            bins[len(bins) - 1] = np.inf
            binning[col] = bins
    return df2, binning


def apply_bins(df, binning):
    df2 = df.copy()
    columns = [col for col in df2.columns if (col != 'CLASS' and col != 'ID')]
    for col in columns:
        labels = list(range(len(binning[col]) - 1))
        df2[col] = pd.cut(df2[col], binning[col], labels=labels)
    return df2

## 1. Define the class RandomForest

In [8]:
# Define the class RandomForest with three functions __init__, fit and predict (after the comments):
#
# Input to __init__: 
# self - the object itself
#
# Output from __init__:
# <nothing>
# 
# This function does not return anything but just initializes the following attributes of the object (self) to None:
# column_filter, imputation, one_hot, labels, model
#
# Input to fit:
# self      - the object itself
# df        - a dataframe (where the column names "CLASS" and "ID" have special meaning)
# no_trees  - no. of trees in the random forest (default = 100)
#
# Output from fit:
# <nothing>
#
# The result of applying this function should be:
#
# self.column_filter - a column filter (see Assignment 1) from df
# self.imputation    - an imputation mapping (see Assignment 1) from df
# self.one_hot       - a one-hot mapping (see Assignment 1) from df
# self.labels        - a (sorted) list of the categories of the "CLASS" column of df
# self.model         - a random forest, consisting of no_trees trees, where each tree is generated from a bootstrap sample
#                      and the number of evaluated features is log2|F| where |F| is the total number of features
#                      (for details, see lecture slides)
#
# Note that the function does not return anything but just assigns values to the attributes of the object.
#
# Hint 1: First create the column filter, imputation and one-hot mappings
#
# Hint 2: Then get the class labels and the numerical values (as an ndarray) from the dataframe after dropping the class labels 
#
# Hint 3: Generate no_trees classification trees, where each tree is generated using DecisionTreeClassifier 
#         from a bootstrap sample (see lecture slides), e.g., generated by np.random.choice (with replacement) 
#         from the row numbers of the ndarray, and where a random sample of the features are evaluated in
#         each node of each tree, of size log2(|F|), where |F| is the total number of features;
#         see the parameter max_features of DecisionTreeClassifier
#
# Input to predict:
# self - the object itself
# df   - a dataframe
# 
# Output from predict:
# predictions - a dataframe with class labels as column names and the rows corresponding to
#               predictions with estimated class probabilities for each row in df, where the class probabilities
#               are the averaged probabilities output by each decision tree in the forest
#
# Hint 1: Drop any "CLASS" and "ID" columns of the dataframe first and then apply column filter, imputation and one_hot
#
# Hint 2: Iterate over the trees in the forest to get the prediction of each tree by the method predict_proba(X) where 
#         X are the (numerical) values of the transformed dataframe; you may get the average predictions of all trees,
#         by first creating a zero-matrix with one row for each test instance and one column for each class label, 
#         to which you add the prediction of each tree on each iteration, and then finally divide the prediction matrix
#         by the number of trees.
#
# Hint 3: You may assume that each bootstrap sample that was used to generate each tree has included all possible
#         class labels and hence the prediction of each tree will contain probabilities for all class labels
#         (in the same order). Note that this assumption may be violated, and this limitation will be addressed 
#         in the next part of the assignment. 

class RandomForest:
    def __init__(self):
        self.column_filter = None
        self.imputation = None
        self.one_hot = None
        self.labels = None
        self.model = None

    def fit(self, df, no_trees=100):
        df2 = df.copy()
        df2, self.column_filter = create_column_filter(df2)
        df2, self.imputation = create_imputation(df2)
        df2, self.one_hot = create_one_hot(df2)

        # Set labels
        self.labels = np.array(pd.Series(df2['CLASS'].astype('category').unique().sort_values()))

        training_labels = df2['CLASS'].values
        df2.drop(['CLASS'], axis='columns', inplace=True)
        feature_names = df2.columns
        # Type casting to int uses floor value. With ceiling value, accuracy improves
        max_features = int(np.log2(df2.columns.size))
        self.model = []

        for i in range(no_trees):
            bootstrap_instances = np.random.choice(df2.shape[0], df2.shape[0], replace=True)
            X = df2.iloc[bootstrap_instances, :].values
            y = training_labels[bootstrap_instances]
            tree = DecisionTreeClassifier(max_features=max_features)
            tree.fit(X, y)
            self.model.append(tree)

    def predict(self, df):
        df2 = df.copy()
        df2.drop(columns=['CLASS'], inplace=True)
        df2 = apply_column_filter(df2, self.column_filter)
        df2 = apply_imputation(df2, self.imputation)
        df2 = apply_one_hot(df2, self.one_hot)
        predictions = pd.DataFrame(np.zeros((df2.index.size, self.labels.size)), columns=self.labels)

        for tree in self.model:
            predictions += tree.predict_proba(df2.values)

        return predictions / len(self.model)
    
# ==============================
# Test your code (leave this part unchanged, except for if auc is undefined)
train_df = pd.read_csv("tic-tac-toe_train.csv")
test_df = pd.read_csv("tic-tac-toe_test.csv")
rf = RandomForest()
t0 = time.perf_counter()
rf.fit(train_df)
print("Training time: {:.2f} s.".format(time.perf_counter()-t0))
test_labels = test_df["CLASS"]
t0 = time.perf_counter()
predictions = rf.predict(test_df)
print("Testing time: {:.2f} s.".format(time.perf_counter()-t0))
print("Accuracy: {:.4f}".format(accuracy(predictions,test_labels)))
print("AUC: {:.4f}".format(auc(predictions,test_labels))) # Comment this out if not implemented in assignment 1
print("Brier score: {:.4f}".format(brier_score(predictions,test_labels))) # Comment this out if not implemented in assignment 1
# ==============================
train_labels = train_df["CLASS"]
predictions = rf.predict(train_df)
print("Accuracy on training set: {0:.4f}".format(accuracy(predictions,train_labels)))
print("AUC on training set: {0:.4f}".format(auc(predictions,train_labels))) # Comment this out if not implemented in assignment 1
print("Brier score on training set: {0:.4f}".format(brier_score(predictions,train_labels))) # Comment this out if not implemented in assignment 1


Training time: 0.33 s.
Testing time: 0.12 s.
Accuracy: 0.9061
AUC: 0.9914
Brier score: 0.1804
Accuracy on training set: 1.0000
AUC on training set: 1.0000
Brier score on training set: 0.0213


In [9]:
# Test your code (leave this part unchanged, except for if auc is undefined)
train_df = pd.read_csv("tic-tac-toe_train.csv")
test_df = pd.read_csv("tic-tac-toe_test.csv")
rf = RandomForest()
t0 = time.perf_counter()
rf.fit(train_df)
print("Training time: {:.2f} s.".format(time.perf_counter()-t0))
test_labels = test_df["CLASS"]
t0 = time.perf_counter()
predictions = rf.predict(test_df)
print("Testing time: {:.2f} s.".format(time.perf_counter()-t0))
print("Accuracy: {:.4f}".format(accuracy(predictions,test_labels)))
print("AUC: {:.4f}".format(auc(predictions,test_labels))) # Comment this out if not implemented in assignment 1
print("Brier score: {:.4f}".format(brier_score(predictions,test_labels))) # Comment this out if not implemented in assignment 1

Training time: 0.35 s.
Testing time: 0.11 s.
Accuracy: 0.9228
AUC: 0.9911
Brier score: 0.1715


In [10]:
train_labels = train_df["CLASS"]
predictions = rf.predict(train_df)
print("Accuracy on training set: {0:.4f}".format(accuracy(predictions,train_labels)))
print("AUC on training set: {0:.4f}".format(auc(predictions,train_labels))) # Comment this out if not implemented in assignment 1
print("Brier score on training set: {0:.4f}".format(brier_score(predictions,train_labels))) # Comment this out if not implemented in assignment 1

Accuracy on training set: 1.0000
AUC on training set: 1.0000
Brier score on training set: 0.0212


### Comment on assumptions, things that do not work properly, etc.


## 2a. Handling trees with non-aligned predictions

In [11]:
# Define a revised version of the class RandomForest with the same input and output as described in part 1 above,
# where the predict function is able to handle the case where the individual trees are trained on bootstrap samples
# that do not include all class labels in the original training set. This leads to that the class probabilities output
# by the individual trees in the forest do not refer to the same set of class labels.
#
# Hint 1: The categories obtained with <pandas series>.cat.categories are sorted in the same way as the class labels
#         of a DecisionTreeClassifier; the latter are obtained by <DecisionTreeClassifier>.classes_ 
#         The problem is that classes_ may not include all possible labels, and hence the individual predictions 
#         obtained by <DecisionTreeClassifier>.predict_proba may be of different length or even if they are of the same
#         length do not necessarily refer to the same class labels. You may assume that each class label that is not included
#         in a bootstrap sample should be assigned zero probability by the tree generated from the bootstrap sample. 
#
# Hint 2: Create a mapping from the complete (and sorted) set of class labels l0, ..., lk-1 to a set of indexes 0, ..., k-1,
#         where k is the number of classes
#
# Hint 3: For each tree t in the forest, create a (zero) matrix with one row per test instance and one column per class label,
#         to which one column is added at a time from the output of t.predict_proba 
#
# Hint 4: For each column output by t.predict_proba, its index i may be used to obtain its label by t.classes_[i];
#         you may then obtain the index of this label in the ordered list of all possible labels from the above mapping (hint 2); 
#         this index points to which column in the prediction matrix the output column should be added to 

class RandomForest:
    def __init__(self):
        self.column_filter = None
        self.imputation = None
        self.one_hot = None
        self.labels = None
        self.model = None

    def fit(self, df, no_trees=100):
        df2 = df.copy()
        df2, self.column_filter = create_column_filter(df2)
        df2, self.imputation = create_imputation(df2)
        df2, self.one_hot = create_one_hot(df2)

        # Set labels
        self.labels = np.array(pd.Series(df2['CLASS'].astype('category').unique().sort_values()))

        training_labels = df2['CLASS'].values
        df2.drop(['CLASS'], axis='columns', inplace=True)
        feature_names = df2.columns
        # Type casting to int uses floor value. With ceiling value, accuracy improves
        max_features = int(np.log2(df2.columns.size))
        self.model = []


        for i in range(no_trees):
            bootstrap_instances = np.random.choice(df2.shape[0], df2.shape[0], replace=True)
            X = df2.iloc[bootstrap_instances, :].values
            y = training_labels[bootstrap_instances]
            tree = DecisionTreeClassifier(max_features=max_features)
            tree.fit(X, y)
            self.model.append(tree)

    def predict(self, df):
        df2 = df.copy()
        df2.drop(columns=['CLASS'], inplace=True)
        df2 = apply_column_filter(df2, self.column_filter)
        df2 = apply_imputation(df2, self.imputation)
        df2 = apply_one_hot(df2, self.one_hot)
        predictions = pd.DataFrame(np.zeros((df2.index.size, self.labels.size)), columns=self.labels)

        for tree in self.model:
            tree_predictions = tree.predict_proba(df2.values)
            for label, i in zip(tree.classes_, range(len(tree.classes_))):
                label_index = np.where(self.labels == label)[0][0]
                predictions[self.labels[label_index]] += tree_predictions[:, i]

        return predictions / len(self.model)

# ==================================================
# Test your code (leave this part unchanged, except for if auc is undefined)

train_df = pd.read_csv("anneal_train.csv")

test_df = pd.read_csv("anneal_test.csv")

rf = RandomForest()

t0 = time.perf_counter()
rf.fit(train_df)
print("Training time: {:.2f} s.".format(time.perf_counter()-t0))

test_labels = test_df["CLASS"]

t0 = time.perf_counter()
predictions = rf.predict(test_df)
print("Testing time: {:.2f} s.".format(time.perf_counter()-t0))

print("Accuracy: {:.4f}".format(accuracy(predictions,test_labels)))
print("AUC: {:.4f}".format(auc(predictions,test_labels))) # Comment this out if not implemented in assignment 1
print("Brier score: {:.4f}".format(brier_score(predictions,test_labels))) # Comment this out if not implemented in assignment 1
# ==================================================

Training time: 0.43 s.
Testing time: 0.64 s.
Accuracy: 0.9443
AUC: 0.9749
Brier score: 0.0979


In [12]:
# Test your code (leave this part unchanged, except for if auc is undefined)
train_df = pd.read_csv("anneal_train.csv")
test_df = pd.read_csv("anneal_test.csv")
rf = RandomForest()
t0 = time.perf_counter()
rf.fit(train_df)
print("Training time: {:.2f} s.".format(time.perf_counter()-t0))
test_labels = test_df["CLASS"]
t0 = time.perf_counter()
predictions = rf.predict(test_df)
print("Testing time: {:.2f} s.".format(time.perf_counter()-t0))
print("Accuracy: {:.4f}".format(accuracy(predictions,test_labels)))
print("AUC: {:.4f}".format(auc(predictions,test_labels))) # Comment this out if not implemented in assignment 1
print("Brier score: {:.4f}".format(brier_score(predictions,test_labels))) # Comment this out if not implemented in assignment 1

Training time: 0.59 s.
Testing time: 0.57 s.
Accuracy: 0.9443
AUC: 0.9717
Brier score: 0.0965


## 2b. Estimate predictive performance using out-of-bag predictions

In [13]:
# Define an extended version of the class RandomForest with the same input and output as described in part 2a above,
# where the results of the fit function also should include:
# self.oob_acc - the accuracy estimated on the out-of-bag predictions, i.e., the fraction of training instances for 
#                which the given (correct) label is the same as the predicted label when using only trees for which
#                the instance is out-of-bag
#
# Hint 1: You may first create a zero matrix with one row for each training instance and one column for each class label
#         and one zero vector to allow for storing aggregated out-of-bag predictions and the number of out-of-bag predictions
#         for each training instance, respectively
#
# Hint 2: After generating a tree in the forest, iterate over the indexes that were not included in the bootstrap sample
#         and add a prediction of the tree to the out-of-bag prediction matrix and update the count vector
#
# Hint 3: Note that the input to predict_proba has to be a matrix; from a single vector (row) x, a matrix with one row
#         can be obtained by x[None,:]
#
# Hint 4: Finally, divide each row in the out-of-bag prediction matrix with the corresponding element of the count vector

class RandomForest:
    def __init__(self):
        self.column_filter = None
        self.imputation = None
        self.one_hot = None
        self.labels = None
        self.model = None
        self.oob_acc = None

    def fit(self, df, no_trees=100):
        df2 = df.copy()
        df2, self.column_filter = create_column_filter(df2)
        df2, self.imputation = create_imputation(df2)
        df2, self.one_hot = create_one_hot(df2)

        # Set labels
        self.labels = np.array(pd.Series(df2['CLASS'].astype('category').unique().sort_values()))

        training_labels = df2['CLASS'].values
        df2.drop(['CLASS'], axis='columns', inplace=True)
        feature_names = df2.columns
        # Type casting to int uses floor value. With ceiling value, accuracy improves
        max_features = int(np.log2(df2.columns.size))
        self.model = []
        oob_predictions = pd.DataFrame(np.zeros((df2.index.size, self.labels.size)), columns=self.labels)
        oob_predictions_count = np.zeros(df2.index.size)
        training_labels = df["CLASS"]

        for i in range(no_trees):
            bootstrap_instances = np.random.choice(df2.shape[0], df2.shape[0], replace=True)
            X = df2.iloc[bootstrap_instances, :].values
            y = training_labels[bootstrap_instances]
            tree = DecisionTreeClassifier(max_features=max_features)
            tree.fit(X, y)
            self.model.append(tree)
            
            # Set out-of-bag predictions
            """ 
                Optimized solution with fast training by using batch predict and batch summation 
                instead of instance by instance 
            """
            oob_indexes = df2.index.difference(bootstrap_instances)
            tmp = tree.predict_proba(df2.iloc[oob_indexes, :].values)

            oob_predictions_count[oob_indexes] += 1
            for label, j in zip(tree.classes_, range(len(tree.classes_))):
                label_index = np.where(self.labels == label)[0][0]
                oob_predictions.loc[oob_indexes, self.labels[label_index]] += tmp[:, j]
            
            
            """ Oringinal solution with slow runtime wrt training time:            
            for j in range(df2.index.size):
                if j in bootstrap_instances:
                    continue
                tmp = tree.predict_proba(df2.iloc[j, :].values.reshape(1, -1))
                
                # This loop is the main culprit for slow runtime
                for label, k in zip(tree.classes_, range(len(tree.classes_))):
                    label_index = np.where(self.labels == label)[0][0]
                    oob_predictions.loc[j, self.labels[label_index]] += tmp[:, k]
                oob_predictions_count[j] += 1
            """ 

        for i in range(oob_predictions.index.size):
            oob_predictions.iloc[i, :] /= oob_predictions_count[i]

        self.oob_acc = accuracy(oob_predictions, training_labels)

    def predict(self, df):
        df2 = df.copy()
        df2.drop(columns=['CLASS'], inplace=True)
        df2 = apply_column_filter(df2, self.column_filter)
        df2 = apply_imputation(df2, self.imputation)
        df2 = apply_one_hot(df2, self.one_hot)
        predictions = pd.DataFrame(np.zeros((df2.index.size, self.labels.size)), columns=self.labels)

        for tree in self.model:
            tree_predictions = tree.predict_proba(df2.values)
            for label, i in zip(tree.classes_, range(len(tree.classes_))):
                label_index = np.where(self.labels == label)[0][0]
                predictions[self.labels[label_index]] += tree_predictions[:, i]

        return predictions / len(self.model)
    
# ============================================
# Test your code (leave this part unchanged, except for if auc is undefined)
train_df = pd.read_csv("anneal_train.csv")
test_df = pd.read_csv("anneal_test.csv")
rf = RandomForest()
t0 = time.perf_counter()
rf.fit(train_df)
print("Training time: {:.2f} s.".format(time.perf_counter()-t0))
print("OOB accuracy: {:.4f}".format(rf.oob_acc))
test_labels = test_df["CLASS"]
t0 = time.perf_counter()
predictions = rf.predict(test_df)
print("Testing time: {:.2f} s.".format(time.perf_counter()-t0))
print("Accuracy: {:.4f}".format(accuracy(predictions,test_labels)))
print("AUC: {:.4f}".format(auc(predictions,test_labels))) # Comment this out if not implemented in assignment 1
print("Brier score: {:.4f}".format(brier_score(predictions,test_labels))) # Comment this out if not implemented in assignment 1
# ============================================
train_labels = train_df["CLASS"]
rf = RandomForest()
rf.fit(train_df)
predictions = rf.predict(train_df)
print("Accuracy on training set: {0:.2f}".format(accuracy(predictions,train_labels)))
print("AUC on training set: {0:.2f}".format(auc(predictions,train_labels)))
print("Brier score on training set: {0:.2f}".format(brier_score(predictions,train_labels)))

Training time: 1.48 s.
OOB accuracy: 0.9510
Testing time: 0.33 s.
Accuracy: 0.9488
AUC: 0.9672
Brier score: 0.1035
Accuracy on training set: 1.00
AUC on training set: 1.00
Brier score on training set: 0.01


In [14]:
# Test your code (leave this part unchanged, except for if auc is undefined)

train_df = pd.read_csv("anneal_train.csv")

test_df = pd.read_csv("anneal_test.csv")

rf = RandomForest()

t0 = time.perf_counter()
rf.fit(train_df)
print("Training time: {:.2f} s.".format(time.perf_counter()-t0))

print("OOB accuracy: {:.4f}".format(rf.oob_acc))

test_labels = test_df["CLASS"]

t0 = time.perf_counter()
predictions = rf.predict(test_df)
print("Testing time: {:.2f} s.".format(time.perf_counter()-t0))

print("Accuracy: {:.4f}".format(accuracy(predictions,test_labels)))
print("AUC: {:.4f}".format(auc(predictions,test_labels))) # Comment this out if not implemented in assignment 1
print("Brier score: {:.4f}".format(brier_score(predictions,test_labels))) # Comment this out if not implemented in assignment 1

Training time: 1.56 s.
OOB accuracy: 0.9488
Testing time: 0.45 s.
Accuracy: 0.9488
AUC: 0.9710
Brier score: 0.1005


In [10]:
train_labels = train_df["CLASS"]
rf = RandomForest()
rf.fit(train_df)
predictions = rf.predict(train_df)
print("Accuracy on training set: {0:.2f}".format(accuracy(predictions,train_labels)))
print("AUC on training set: {0:.2f}".format(auc(predictions,train_labels)))
print("Brier score on training set: {0:.2f}".format(brier_score(predictions,train_labels)))

Accuracy on training set: 1.00
AUC on training set: 1.00
Brier score on training set: 0.01


### Comment on assumptions, things that do not work properly, etc.